# Naive Bayes Implementation

Referencias:
- http://artificiencia.com/aprenda/algoritmo-naive-bayes/
- https://chrisalbon.com/machine_learning/naive_bayes/naive_bayes_classifier_from_scratch/ (DONE)
- https://hackinganalytics.com/2016/10/13/desvendando-o-classificador-naive-bayes/ (DONE)
- https://towardsdatascience.com/naive-bayes-in-machine-learning-f49cc8f831b4


1. Dado um dataset, calcular probabilidade de cada classe
    - P(Jogar = Sim) = 9/14;
    - p(Jogar = Nao) = 5/14;
2. Calcular probabilidade de cada atributo
    - P(Genero = Sol | Target = Sim) = 2/9;
    - Continua...
    
    
## Dúvidas
R: Sobre o treinamento, o que é o treinamento do Navie Bayes.<br>
R: Treinamento é as `probabilidade condicional`

In [1]:
import pandas as pd
import numpy as np

dataset = pd.read_csv('dataset/dataset.csv')
dataset

,Genero,Idade,Escolaridade,Profissao,Target
0,F,a - Ate 25 anos,Fundamental,b,0
1,M,a - Ate 25 anos,Medio,d,1
2,F,c - 36 a 45 anos,Fundamental,a,1
3,M,d - 46 a 55 anos,Fundamental,a,0
4,F,c - 36 a 45 anos,Fundamental,b,1
5,F,b - 26 a 35 anos,Fundamental,b,0
6,F,a - Ate 25 anos,Fundamental,b,0
7,F,e - Mais 56 anos,Superior,b,1
8,F,a - Ate 25 anos,Superior,b,1
9,M,b - 26 a 35 anos,Fundamental,a,1


### Funções auxiliares
- P(A | B):
    - Função auxiliar para fazer `probabilidade condicional`
- P(A):
    - Função auxiliar para fazer `probabilidade condicional`

In [2]:
def p_given_q(p, q, val_p, val_q):
    p_f_zero = dataset[(dataset[p]==val_p) & (dataset[q]==val_q)][p].count()/dataset[p][dataset[q]==val_q].count()
    return p_f_zero

def p_q(p,val_p):
    return dataset[p][dataset[p]==val_p].count()/dataset[p].count()

#p_given_q('Weather','Target','sunny',1)

### Probabilidade de Target ser 0 ou 1

In [3]:
# Counters
n_tg_zero = dataset['Target'][dataset['Target']==0].count()
n_tg_one = dataset['Target'][dataset['Target']==1].count()

n_all = dataset['Target'].count()

In [4]:
p_tg_zero = n_tg_zero/n_all
p_tg_one = n_tg_one/n_all

### Listas com possibilidades

In [5]:
#Training

list_columns = list(dataset)
list_columns.pop()
list_value_tg = list(dataset['Target'].unique())
list_results = list()

for column in list_columns:
    for value_col in list(dataset[column].unique()):
        for value_tg in list_value_tg:
            list_results.append((column+'='+str(value_col)+' dado Target='+ str(value_tg), p_given_q(column,'Target',value_col,value_tg)))
list_results

[('Genero=F dado Target=0', 0.5131578947368421),
 ('Genero=F dado Target=1', 0.6036269430051814),
 ('Genero=M dado Target=0', 0.4868421052631579),
 ('Genero=M dado Target=1', 0.3963730569948187),
 ('Idade=a - Ate 25 anos  dado Target=0', 0.40350877192982454),
 ('Idade=a - Ate 25 anos  dado Target=1', 0.15025906735751296),
 ('Idade=c - 36 a 45 anos dado Target=0', 0.2149122807017544),
 ('Idade=c - 36 a 45 anos dado Target=1', 0.2538860103626943),
 ('Idade=d - 46 a 55 anos dado Target=0', 0.04824561403508772),
 ('Idade=d - 46 a 55 anos dado Target=1', 0.16839378238341968),
 ('Idade=b - 26 a 35 anos dado Target=0', 0.30701754385964913),
 ('Idade=b - 26 a 35 anos dado Target=1', 0.29404145077720206),
 ('Idade=e - Mais 56 anos dado Target=0', 0.02631578947368421),
 ('Idade=e - Mais 56 anos dado Target=1', 0.133419689119171),
 ('Escolaridade=Fundamental dado Target=0', 0.5394736842105263),
 ('Escolaridade=Fundamental dado Target=1', 0.4896373056994819),
 ('Escolaridade=Medio dado Target=0', 

### Formula para predições
MAP(h) = max (P (o|h) x P(h))

In [8]:
# Calcule predicts
count_correct = 0
count_incorrect = 0

for index, row in dataset.iterrows():
    classes = list()
    for value_tg in list_value_tg:
        prob = p_q('Target',value_tg)
        for column in list_columns:
            prob *= p_given_q(column,'Target',row[column],value_tg)
        classes.append((value_tg, prob))
    #Result
    result = max(classes,key=lambda p: p[1])
    if result[0] == row['Target']:
        count_correct += 1
    else:
        count_incorrect += 1

print(count_correct)
print(count_incorrect)
        

791
209


In [9]:
print(n_tg_one)
print(n_tg_zero)

772
228
